In [10]:
import pandas as pd
import requests
from datetime import datetime
from time import sleep

# Load your data
df = pd.read_csv("Map_Overview_data.csv")

# Your Visual Crossing API key
API_KEY = "Paste API quey here"

# Cache for already-queried (rounded_lat, rounded_lon, date) tuples
weather_cache = {}

def extract_datetime(row):
    try:
        date_str = row['Collision_Date_Tooltip'].split('@')[0].strip()
        date_obj = datetime.strptime(date_str, "%A, %B %d, %Y")
        date_fmt = date_obj.strftime("%Y-%m-%d")
        time_str = row['Collision_Time_adj'].strip()
        if time_str.lower() == "unknown":
            return date_fmt, None
        return date_fmt, time_str
    except:
        return None, None

def get_weather(lat, lon, date_str, time_str):
    base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    location = f"{lat},{lon}"
    url = f"{base_url}{location}/{date_str}?key={API_KEY}&unitGroup=us&include=hours"

    while True:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                break
            elif response.status_code == 429:
                print(" → Hit rate limit (429). Sleeping for 60 seconds...")
                sleep(60)
            else:
                return f"HTTP {response.status_code}"
        except Exception as e:
            return f"Exception: {e}"

    try:
        data = response.json()
        hours = data['days'][0].get('hours', [])
        if not hours:
            return "No hourly data"

        # Round to nearest hour
        crash_time = datetime.strptime(time_str, "%H:%M")
        crash_hour = round(crash_time.hour + crash_time.minute / 60)

        for hour_data in hours:
            api_hour = int(hour_data['datetime'].split(":")[0])
            if api_hour == crash_hour:
                return hour_data.get('conditions', 'No condition info')

        return "No matching hour"
    except Exception as e:
        return f"Exception: {e}"

# Collect weather conditions
weather_conditions = []

for i, row in df.iterrows():
    lat, lon = row['Latitude_adj'], row['Longitude_adj']
    date_str, time_str = extract_datetime(row)
    cache_key = (date_str, round(lat, 2), round(lon, 2))

    print(f"[{i+1}/{len(df)}] Getting weather for {date_str} {time_str or '[unknown]'} at ({lat},{lon})")

    if date_str and time_str:
        if cache_key in weather_cache:
            condition = weather_cache[cache_key]
            print(" → (cached)")
        else:
            condition = get_weather(lat, lon, date_str, time_str)
            weather_cache[cache_key] = condition
    else:
        condition = "Missing or invalid time/date"

    print(f" → Weather: {condition}")
    weather_conditions.append(condition)
    
    # Save partial results every 50 rows
    if i > 0 and i % 50 == 0:
        temp_df = df.iloc[:i+1].copy()
        temp_df["Weather_Fetched"] = weather_conditions
        temp_df.to_csv("Map_Overview_with_Weather_partial.csv", index=False)
        print("Saved checkpoint: Map_Overview_with_Weather_partial.csv")

    sleep(2)  # More conservative delay

# Final save
df["Weather_Fetched"] = weather_conditions
df.to_csv("Map_Overview_with_Weather.csv", index=False)
print("✅ Done! File saved as Map_Overview_with_Weather.csv")

[1/621] Getting weather for 2025-05-17 10:30 at (35.38937,-89.41693)
 → Weather: Clear
[2/621] Getting weather for 2025-05-18 [unknown] at (35.33464,-89.549)
 → Weather: Missing or invalid time/date
[3/621] Getting weather for 2025-05-10 0:19 at (35.90948,-83.18574)
 → Weather: Partially cloudy
[4/621] Getting weather for 2025-05-09 15:25 at (35.19159,-89.97634)
 → Weather: Overcast
[5/621] Getting weather for 2025-05-08 21:30 at (36.15652,-86.82544)
 → Weather: Overcast
[6/621] Getting weather for 2025-05-08 11:43 at (36.18,-85.996)
 → Weather: Partially cloudy
[7/621] Getting weather for 2025-04-30 13:59 at (36.13838,-85.57062)
 → Weather: Rain, Partially cloudy
[8/621] Getting weather for 2025-04-29 16:27 at (35.72703,-88.60495)
 → Weather: Clear
[9/621] Getting weather for 2025-04-28 7:07 at (36.02828,-87.17494)
 → Weather: Clear
[10/621] Getting weather for 2025-04-26 1:26 at (36.18206,-86.15224)
 → Weather: Partially cloudy
[11/621] Getting weather for 2025-04-25 16:46 at (35.315